In [3]:
!pwd

/home/tucker/sabbatical/predict_bert_embeddings/char_lm/char_seq_to_masked_emb


In [30]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint, RunOne
from yaml import safe_load
import torch.nn as nn

In [62]:
exphash = '561fdb2f9'
exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

ds = InitDataset(h)


In [63]:
#h['hyperparameters']['manual_attention'] = True
model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[12:32:25] loading configuration file bert_distil_dense_out/config.json


[12:32:25] loading weights file bert_distil_dense_out/pytorch_model.bin


[12:32:27] Loaded model: <All keys matched successfully>


In [64]:
h

{'dataset': 'imdb_tok96_char_encoded384charidx2.pt',
 'dataset_split': [0.5, 0.1],
 'experiment_hash': '561fdb2f9',
 'experiment_name': 'testing_bert_charcloze_to_dense_tok',
 'finished_date': '2020-06-18 12:29:50.640342',
 'hyperparameters': {'attn_conv.kernel|filter_sizes': [[6, 256],
   [5, 128],
   [4, 128],
   [1, 1]],
  'batch_size': 32,
  'bert_checkpoint': 'bert_distil_dense_out',
  'char_embedding_size': 8,
  'char_vocab_size': 70,
  'conv_activation': 'relu',
  'epochs': 0,
  'freeze_modules': ['bert.transformer.layer.0',
   'bert.transformer.layer.1',
   'bert.transformer.layer.2',
   'bert.transformer.layer.3',
   'bert.transformer.layer.4',
   'bert.transformer.layer.5',
   'bert.embeddings',
   'vocab_layer_norm',
   'vocab_transform',
   'vocab_projector',
   'embedding_predict'],
  'input_type': 'char_masked',
  'learning_rate': 0.0001,
  'loss_fn': 'xentropy',
  'lr_step_size': 1000,
  'manual_attention': False,
  'mask_chars': True,
  'model_checkpoint': 'models/70757

In [65]:
h['dataset_split'][1] = .01

In [66]:
val_loader = DataLoader(ds[1], batch_size=64, shuffle=False)


In [67]:
ds[0][0]['label_ids'].shape

torch.Size([2])

In [68]:
ds[0][0]['label_ids']

tensor([ 101, 2005])

In [69]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [70]:
xent = nn.CrossEntropyLoss()
def xentropy_loss_fn(output, labels):
  output = output.contiguous()  
  return xent(output.view(-1, output.size(-1)), labels.view(-1))

def top1_acc_fn(output, labels):
  top = output.argmax(-1)
  right = top==labels
  return right.float()

def Validate(val_loader, model):
  device = next(model.parameters()).device
  cum_acc = 0
  cum_xent_loss = 0

  rights_ = []
  inputs_ =[]
  labels_ =[]
  maxes_ =[]
  for i, data in enumerate(val_loader):
    data = {k: d.to(device) for k,d in data.items()}
    inputs = data['input_ids']
    labels = data['label_ids']
    inputs_ += Torch2Py(inputs)
    labels_ += Torch2Py(labels)
    
    with torch.no_grad():
      
      model.eval()
      outputs = model(inputs)
      maxes_ += Torch2Py(outputs.argmax(-1))
      xent_loss = xentropy_loss_fn(outputs, labels)
      cum_xent_loss += xent_loss      

      top1_acc = top1_acc_fn(outputs, labels)
      rights_ += Torch2Py(top1_acc)
      cum_acc += top1_acc.mean()
      break

  steps = i+1
  print(cum_acc/steps, cum_xent_loss.detach()/steps)
  return rights_, inputs_, labels_, maxes_,



In [71]:
right, inputs, labels, maxes = Validate(val_loader, model)

tensor(0.4922, device='cuda:0') tensor(3.9828, device='cuda:0')


In [72]:
np.array(inputs)[:10, :15]

array([[32, 46, 55, 62, 33,  1, 58, 57,  1, 44, 63,  1, 63, 51, 48],
       [32, 46, 55, 62, 33,  1, 62, 58,  1, 51, 52, 62,  1, 51, 48],
       [32, 46, 55, 62, 33,  1, 44, 55, 52, 48, 57, 62,  1, 59, 65],
       [32, 46, 55, 62, 33,  1, 62, 51, 48,  1, 52, 62,  1, 62, 48],
       [32, 46, 55, 62, 33,  1, 51, 48, 55, 59,  1, 51, 48, 61,  1],
       [32, 46, 55, 62, 33,  1, 11, 11, 11, 11,  1, 63, 51, 52, 62],
       [32, 46, 55, 62, 33,  1, 59, 61, 52, 62, 58, 57, 48, 61, 62],
       [32, 46, 55, 62, 33,  1, 63, 51, 48,  1, 59, 61, 48, 56, 52],
       [32, 46, 55, 62, 33,  1, 63, 51, 48, 52, 61,  1, 58, 59, 52],
       [32, 46, 55, 62, 33,  1, 58, 49,  1, 63, 51, 48,  1, 57, 48]])

In [73]:
len(right), len(inputs), len(labels), len(maxes),

(64, 64, 64, 64)

In [74]:
model.embedded_chars[0].shape

torch.Size([96, 768])

In [75]:
char_to_idx_map = torch.load('../data/char_to_idx_map2.pt')

In [76]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}

In [77]:
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()

In [78]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [79]:
def GetToks(encoded):
  s = ""
  for c in encoded:
    s+= " " + idx_to_tok_map[c]
  return s

In [80]:
torch.tensor(inputs[:1])

tensor([[32, 46, 55, 62, 33,  1, 58, 57,  1, 44, 63,  1, 63, 51, 48,  1, 62, 44,
         56, 48,  1, 63, 52, 56, 48, 15,  1, 63, 51, 48, 61, 48,  1, 47, 58, 48,
         62, 57,  8, 63,  1, 62, 48, 48, 56,  1, 63, 58,  1, 45, 48,  1, 44, 57,
         68,  1, 46, 58, 57, 57, 48, 46, 63, 52, 58, 57,  1, 45, 48, 63, 66, 48,
         48, 57,  1, 63, 51, 48,  1, 63, 66, 58,  1,  9, 52,  1, 56, 52, 62, 62,
          1, 63, 51, 48,  1, 32, 56, 33,  1, 48, 59, 52, 62, 58, 47, 48, 13,  1,
         62, 58,  1, 52,  1, 47, 58, 57,  8, 63,  1, 54, 57, 58, 66,  1, 52, 49,
          1, 63, 51, 48, 68,  1, 51, 44, 65, 48,  1, 56, 48, 57, 63, 52, 58, 57,
         48, 47,  1, 63, 51, 48,  1, 46, 58, 57, 57, 48, 46, 63, 52, 58, 57, 13,
          1, 45, 64, 63,  1, 49, 61, 58, 56,  1, 63, 51, 48,  1, 36, 57, 47,  1,
         48, 59, 52, 62, 58, 47, 48,  1, 63, 58,  1, 63, 51, 48,  1, 40, 63, 51,
         13,  1, 63, 51, 48, 61, 48,  1, 52, 62,  1, 57, 58, 63, 51, 52, 57, 50,
          1, 56, 48, 57, 63,

In [57]:
model.char_embedder.tokens_start_attn(torch.tensor(inputs).cuda()[:1])

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

In [81]:
chrs = []
for i in inputs:
  chrs.append(GetChars(i))

In [82]:
chrs[:3]

['[cls] on at the same time. there doesn\'t seem to be any connection between the two (i miss the [m] episode, so i don\'t know if they have mentioned the connection, but from the 2nd episode to the 6th, there is nothing mentioning the connection between the 2 "story") this series deserves to become a regular next season or_______________________________________________________________',
 "[cls] so his heroic effort to win big for the south never really became my own goal. i couldn't find my interest in his skimpy story if i had a civil war ree-[m]acter sitting next to me. scene after scene of audie trying to pull it together, and it can only end one way. ...in his triumph over his_______________________________________________________________________________________",
 '[cls] aliens pvt hudson & the unwilling tech guy ("game over man", vs. "i\'m not going out there") the black hole cowardly dude gets what\'s coming after stealing the escape pod leviathan: sunken cargo ship with danger

In [134]:
n = 3
attn = (model.char_embedder.char_block_attn[:n]>0).float()
attn = model.char_embedder.sb_module.switch_input[:3]

In [96]:
rb = (torch.tensor(inputs)<=32)[:n].float().cuda()

In [131]:
inputs = torch.tensor(inputs)

<ipython-input-131-96c00cc89c27>:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [132]:

spaces = torch.where(inputs==1) # TODO fragile. Alls non alphanumerical characters in char2idxmap2
act = torch.zeros_like(inputs)

# TODO clean and include other special chars.
act[spaces[0], torch.clamp(spaces[1]+1, max=act.shape[-1]-1)] = 1
special_chars = torch.where((inputs<=32) & (inputs!=1))
act[special_chars] = 1
act[torch.where(act==0)]=0
act[:, 0] = 1
rb = act.clone().cpu()[:n]

In [135]:
import plotly.figure_factory as ff
hm = ff.create_annotated_heatmap(
  z=attn.detach().cpu().numpy(), annotation_text=chrs[:n],
  text=attn.detach().cpu().numpy(), hoverinfo='text')

plotly.offline.plot(hm, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [136]:
import plotly.figure_factory as ff
hm = ff.create_annotated_heatmap(
  z=rb.numpy(), annotation_text=chrs[:n],
  text=attn.detach().cpu().numpy(), hoverinfo='text')

plotly.offline.plot(hm, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [120]:
rb

tensor([[1, 0, 0,  ..., 1, 1, 1],
        [1, 0, 0,  ..., 1, 1, 1],
        [1, 0, 0,  ..., 1, 1, 1]])

In [123]:
acc = (rb-attn.cpu()).float()

In [124]:
import plotly.figure_factory as ff
hm = ff.create_annotated_heatmap(
  z=acc.cpu(), annotation_text=chrs[:n],
  text=attn.detach().cpu().numpy(), hoverinfo='text')

plotly.offline.plot(hm, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [87]:
d = ds[2][0]

In [33]:
inps = d['input_ids'].unsqueeze(0)

In [34]:
torch.sort(torch.randperm(100)[:5])[0]

tensor([ 0,  7, 17, 22, 78])

In [35]:
GetChars(Torch2Py(inps)[0])

'[cls] involves secondary institutions -- the police, psychiatrists, or, quien sabe?, the national security [m]. in the case of marie/audrey[m]robbie/etc., all three systems failed for a long time, simply because it seemed so hard to believe that a woman would kill her husband and try to poison her own child [m] insurance that was no more than modest. and[m]_________________________'

In [36]:
GetToks(Torch2Py(d['label_ids']))

' [CLS] agency / for ,'

In [37]:
outs = model(d['input_ids'].unsqueeze(0).cuda())

In [38]:
GetToks(Torch2Py(outs.argmax(-1)[0]))

' [CLS] guard for for ,'

In [39]:
GetToks(Torch2Py(d['mask_idxs']))

' [PAD] [unused22] [unused31] [unused68] [unused78]'

In [40]:
d['input_ids'].cuda()

tensor([32, 46, 55, 62, 33,  1, 52, 57, 65, 58, 55, 65, 48, 62,  1, 62, 48, 46,
        58, 57, 47, 44, 61, 68,  1, 52, 57, 62, 63, 52, 63, 64, 63, 52, 58, 57,
        62,  1, 14, 14,  1, 63, 51, 48,  1, 59, 58, 55, 52, 46, 48, 13,  1, 59,
        62, 68, 46, 51, 52, 44, 63, 61, 52, 62, 63, 62, 13,  1, 58, 61, 13,  1,
        60, 64, 52, 48, 57,  1, 62, 44, 45, 48, 22, 13,  1, 63, 51, 48,  1, 57,
        44, 63, 52, 58, 57, 44, 55,  1, 62, 48, 46, 64, 61, 52, 63, 68,  1, 32,
        56, 33, 15,  1, 52, 57,  1, 63, 51, 48,  1, 46, 44, 62, 48,  1, 58, 49,
         1, 56, 44, 61, 52, 48, 16, 44, 64, 47, 61, 48, 68, 32, 56, 33, 61, 58,
        45, 45, 52, 48, 16, 48, 63, 46, 15, 13,  1, 44, 55, 55,  1, 63, 51, 61,
        48, 48,  1, 62, 68, 62, 63, 48, 56, 62,  1, 49, 44, 52, 55, 48, 47,  1,
        49, 58, 61,  1, 44,  1, 55, 58, 57, 50,  1, 63, 52, 56, 48, 13,  1, 62,
        52, 56, 59, 55, 68,  1, 45, 48, 46, 44, 64, 62, 48,  1, 52, 63,  1, 62,
        48, 48, 56, 48, 47,  1, 62, 58, 

In [41]:
GetToks(Torch2Py(model(inps.cuda()).argmax(-1)[0]))

' [CLS] guard for for ,'

In [42]:
with torch.no_grad():
  a = ((model.embedded_chars[-1, 0:8, :].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [20]:
model.embedded_chars[-1, 0:20, :].unsqueeze(1).shape

torch.Size([20, 1, 768])

In [21]:
with torch.no_grad():
  a = (model.embedded_chars[-1, 0:20, :].squeeze(1)@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)

In [25]:
print(GetChars(inputs[1]))

[cls] rat-pelts, is [m]. he scarcely looks human, [m] shadow-image of the successful doctor, yukio. [m]tekichi represents the oppression of japan's violent, disordered-past from the eras of the shogun and the samurai[m] yukio represents the emergence of a modern japan, with his work as a doctor and his bourgeois life with his new-wife[m] rin. he to[m] helping cure the______________


In [26]:
GetChars(Torch2Py(ds[0][0]['input_ids']))

'[cls] above. how original!). and pacino is [m] by very pretty (and [m]) girls who are in love [m] him. how pathetic. if you really like the "real[m] al pacino, don\'t even think to watch this film. you might be pushed to think that extraterrestrials [m] replaced him with a bad copy. al pacino is one of [m] few__________________________________________________________________________'

In [27]:
print(GetToks(maxes[1]))

 ) " 7 ) is unknown . he hardly looks human " ) shadow 15 of the young doctor ) ##yuki ) he represents the oppression of japan . violent ) disorder 15 from the birth of the earth and the samurai ##yuki represents the emergence of a modern japan , with his work as a doctor and his bourgeois life with his friend / ##s . he is help cure the [ [ [ [ [ [ , , , ... ... ... ... ... ... ##oh ##oh ##oh ##oh ##oh ##oh ##oh ##oh ##o


In [86]:
j = 0
inp = inputs[4][j:j+18]
GetChars(inp), inp

('[cls] [m](it remin',
 [32, 46, 55, 62, 33, 1, 32, 56, 33, 9, 52, 63, 1, 61, 48, 56, 52, 57])

In [87]:
inp = torch.tensor([[50, 64, 52, 63, 44, 61,  1, 35, 41, 40, 35,  1, 58, 45, 62, 46, 48, 57]])

In [88]:
out = model.char_embedder.tokens_to_emb(inp.cuda())

In [89]:
#next(model.char_embedder.tokens_to_emb.named_parameters())

In [90]:
out[0][:4]

tensor([ 0.0719, -0.1866,  0.1774,  0.5457], device='cuda:0',
       grad_fn=<SliceBackward>)

In [91]:
word = (out@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)

In [92]:
model.bert.distilbert.embeddings.word_embeddings.weight.data[2858][:10]

tensor([ 0.0209, -0.0467, -0.0392, -0.0345,  0.0150, -0.0398, -0.0696, -0.0479,
        -0.0290, -0.0037], device='cuda:0')

In [93]:
word

tensor([2858], device='cuda:0', grad_fn=<NotImplemented>)

In [94]:
word = ((out-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [274]:
w = '[cls] guitars [m] some ot [m]'
thing = []
for c in w :
  thing.append(char_to_idx_map[c])
print(len(thing))
inp = torch.tensor(thing).unsqueeze(0)

29


In [248]:
LoadCheckpoint(model.char_embedder, '../single_emb_pred/models/c26917206')

[22:44:05] Loaded model: <All keys matched successfully>


In [275]:
out = model(inp.cuda())

In [276]:
out = model.embedded_chars

In [239]:
out = model.char_embedder(inp.cuda())

In [240]:
out.shape

torch.Size([1, 96, 768])

In [295]:
.005**2

2.5e-05

In [294]:
torch.nn.MSELoss(reduction='none')(torch.tensor([[.005, 0.0, 0.0]]), torch.tensor([[0.0, 0.0, 0.0]]))

tensor([[2.5000e-05, 0.0000e+00, 0.0000e+00]])

In [280]:
word = ((out[0, :4].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [278]:
word = (out@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)[0]

In [281]:
GetToks(Torch2Py(word))

' [CLS] guitars [MASK] some'

In [ ]:
' [CLS] made of stories a where every story is great ; that'

In [42]:
a.shape

torch.Size([8])

In [24]:
print(GetToks(Torch2Py(a)))

 [CLS] retired / it doesn in matter if


In [64]:
print(GetToks(Torch2Py(a)))

 comedy at at at at at now at at at at at at at now at at comedy at at


In [ ]:
p

In [25]:
embeddings = ds.data['embeddings']

AttributeError: 'tuple' object has no attribute 'data'

In [26]:
embeddings.norm(2,-1).mean()

NameError: name 'embeddings' is not defined

In [252]:
embedding_noise = .25

In [253]:
r = torch.normal(mean=embedding_noise, std=1, size=embeddings.size()).cuda()
norm_factor = torch.norm(r, 2, -1).mean()/embedding_noise
normalized_noise = r/norm_factor
new = embeddings + normalized_noise

In [254]:
((new-embeddings)**2).mean()

tensor(8.1433e-05, device='cuda:0')

In [255]:
torch.sqrt((new-embeddings)**2).mean()

tensor(0.0072, device='cuda:0')

In [217]:
n = new.norm(2,-1)

In [31]:
def MakeHist(h, name=None):
  y = h[0]
  x= h[1]
  return {
    'name': name,
    'type': 'bar',                                                                                                                   'x': x,
          'marker': dict(opacity=.7),
   'y': y
  }

In [30]:
dat = MakeHist(np.histogram(n, bins=30), name='toks')

fig = fig = go.Figure(data=[dat], layout={'barmode':'overlay'})
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


NameError: name 'n' is not defined

In [22]:
from experiment import SwitchboardAttention

In [23]:
from experiment import step_with_sigmoid_grad

In [24]:
step_with_sigmoid_grad(torch.tensor([1]))

tensor([1.])

In [25]:
sba = SwitchboardAttention(6, 'step_with_sigmoid_grad')

In [26]:
sba(torch.tensor([[1,1,1,1,1]]).float()*100)

tensor([[[1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]])

In [27]:
attn = torch.nn.Sigmoid()(model.char_embedder.char_block_attn)

In [31]:
hm = go.Heatmap(z=attn[:100].detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [29]:
sb = model.char_embedder.switchboard[0]

In [30]:
hm = go.Heatmap(z=sb.detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [76]:
sb.shape


torch.Size([96, 384])

In [76]:
ds = CharClozeToDenseTokEmbDataset('imdb_tok96_char_encoded384.pt', num_masks=20)

In [77]:
char_to_idx_map = torch.load('../data/char_to_idx_map.pt')

In [78]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}

In [82]:
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()

In [83]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [89]:
def GetToks(encoded):
  s = ""
  for i, c in enumerate(encoded):
    s+= " %d."%i + idx_to_tok_map[c]
  return s

In [27]:
print(GetChars(Torch2Py(ds[0]['input_ids'])))
print(GetChars(Torch2Py(ds[0]['original'])))
print(GetToks(Torch2Py(ds[0]['tokens'])))

TypeError: list indices must be integers or slices, not str

In [61]:
ds[0] 

tensor(76)
tensor([[ 72,  76],
        [ 92,  95],
        [ 96, 101],
        [114, 115],
        [116, 122],
        [122, 123],
        [126, 130],
        [137, 138],
        [139, 142],
        [145, 150],
        [162, 163],
        [168, 171],
        [192, 196],
        [209, 213],
        [220, 221],
        [227, 233],
        [234, 241],
        [252, 253],
        [253, 254],
        [269, 272]], dtype=torch.int16)


{'input_ids': tensor([34, 42, 51, 58, 36,  1, 48,  1, 40, 43, 52, 48, 59, 13,  1, 59, 47, 44,
          1, 46, 57, 44, 40, 59,  1, 52, 40, 49, 54, 57, 48, 59, 64,  1, 54, 45,
          1, 45, 48, 51, 52, 58,  1, 57, 44, 51, 44, 40, 58, 44, 43,  1, 41, 44,
         45, 54, 57, 44,  1, 58, 40, 64,  1, 18, 26, 20, 20,  1, 40, 57, 44,  1,
         34, 38, 36,  1, 53, 54, 59,  1, 45, 54, 57,  1, 52, 44, 15,  1, 54, 45,
          1, 34, 38, 36,  1, 34, 38, 36,  1, 54, 57,  1, 58, 54,  1,  3, 52, 40,
         49, 54, 57, 34, 38, 36,  1, 34, 38, 36, 34, 38, 36,  1, 48,  1, 34, 38,
         36,  1, 61, 48, 44, 62, 44, 43, 34, 38, 36,  1, 34, 38, 36,  1, 48,  1,
         34, 38, 36,  1,  9, 59, 47, 44,  1, 42, 57, 54, 62, 43, 10, 34, 38, 36,
          1, 40, 53, 43,  1, 34, 38, 36,  1, 62, 44, 57, 44,  1, 61, 44, 57, 64,
          1, 46, 54, 54, 43,  1,  9, 59, 47, 44,  1, 34, 38, 36,  1, 42, 54, 52,
         52, 40, 53, 43,  1, 40, 53, 43,  1, 34, 38, 36,  1, 51, 48, 46, 47, 59,
         58, 34

In [7]:
ds.data['token_start_offsets'].shape

torch.Size([200091, 96, 2])

In [39]:
original = ds.data['chars_encoded'][3]
offsets = ds.data['token_start_offsets'][3]
tokens = ds.data['token_ids'][3]
num_toks = (tokens!=0).sum()
rand_toks = torch.randperm(num_toks)
offsets = offsets[rand_toks[:ds.num_masks]]
sorted_offsets = torch.sort(offsets, 0)[0]
segments_to_keep = torch.cat([ds.beg, sorted_offsets.view(-1), ds.end], 0).view(-1,2)

slices = []
char_mask = torch.tensor([34,38,36]).char().to('cuda')
for start, end in segments_to_keep:
  char_slice = original[start:end]
  slices.append(char_slice)
  slices.append(char_mask)

masked_seq = torch.cat(slices, 0)
pad_size = max(0, len(original)-len(masked_seq))
pad = torch.tensor([0]).expand(pad_size).char().to('cuda')
masked_seq = torch.cat([masked_seq, pad], 0)[:len(original)]

5

In [22]:
pad_size

5

In [322]:
original ds.data['token_start_offsets'][0]

AttributeError: 'Tensor' object has no attribute 'randperm'

In [1]:
import torch

In [2]:
a = torch.tensor([[.1,.2,.3,.4,.5]])
b = torch.tensor([[.5,.4,.3,.2,.1]])
c = torch.tensor([[.1,.1,.1,.1,.5]])

In [4]:
def nll(teacher, student):
  return -1*(teacher*torch.log(student)).sum()

In [9]:
nll(b,c)

tensor(3.2929)

In [89]:
class StepWithSigmoidGrad(torch.autograd.Function):
    @staticmethod
    def forward(self,x):
        self.x = x
        return (x > 0).float()

    @staticmethod
    def backward(self,grad_output):
        sig = torch.nn.Sigmoid()(self.x)
        return sig*(1-sig)

class StepWithHardGrad(torch.autograd.Function):
    @staticmethod
    def forward(self,x):
        self.high = (x>-.5) & (x<.5)
        return (x > 0).float()

    @staticmethod
    def backward(self,grad_output):
        grad_input = grad_output.clone()
        print(self.high)
        grad_input[self.high] = 1
        grad_input[~self.high] = 0
        return grad_input
      

In [96]:
f = StepWithHardGrad

In [97]:
vals = []
for i in torch.arange(-2,2,.1):
  t = i.clone().detach()
  t.requires_grad =True
  v = f.apply(t)
  v.backward()
  vals.append((t.item(), v.item(), t.grad.item()))

tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)


In [98]:
vals

[(-2.0, 0.0, 0.0),
 (-1.899999976158142, 0.0, 0.0),
 (-1.7999999523162842, 0.0, 0.0),
 (-1.7000000476837158, 0.0, 0.0),
 (-1.600000023841858, 0.0, 0.0),
 (-1.5, 0.0, 0.0),
 (-1.399999976158142, 0.0, 0.0),
 (-1.2999999523162842, 0.0, 0.0),
 (-1.2000000476837158, 0.0, 0.0),
 (-1.100000023841858, 0.0, 0.0),
 (-1.0, 0.0, 0.0),
 (-0.8999999761581421, 0.0, 0.0),
 (-0.800000011920929, 0.0, 0.0),
 (-0.699999988079071, 0.0, 0.0),
 (-0.6000000238418579, 0.0, 0.0),
 (-0.5, 0.0, 0.0),
 (-0.4000000059604645, 0.0, 1.0),
 (-0.30000001192092896, 0.0, 1.0),
 (-0.20000000298023224, 0.0, 1.0),
 (-0.10000000149011612, 0.0, 1.0),
 (0.0, 0.0, 1.0),
 (0.10000000149011612, 1.0, 1.0),
 (0.20000000298023224, 1.0, 1.0),
 (0.30000001192092896, 1.0, 1.0),
 (0.4000000059604645, 1.0, 1.0),
 (0.5, 1.0, 0.0),
 (0.6000000238418579, 1.0, 0.0),
 (0.699999988079071, 1.0, 0.0),
 (0.800000011920929, 1.0, 0.0),
 (0.8999999761581421, 1.0, 0.0),
 (1.0, 1.0, 0.0),
 (1.100000023841858, 1.0, 0.0),
 (1.2000000476837158, 1.0, 0.0),

In [99]:
t,v,g = zip(*vals)

In [100]:
dat1 = {'type':'scatter', 'x':t, 'y':v}
dat2 = {'type':'scatter', 'x':t, 'y':g}

In [101]:
fig = fig = go.Figure(data=[dat1,dat2])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


'/home/tucker/Downloads/imdb_size_distci.html'

In [2]:
# plotly rescales colors in an incovenient way - manually calculating colors.
class rgb():
  def __init__(self, r, g, b):
    self.r = int(round(r))
    self.g = int(round(g))
    self.b = int(round(b))
    
  def __str__(self):
    return "rgb(%d, %d, %d)" % (self.r, self.g, self.b)
  
  def interpolate(self, other_color, weight):
    # returns a weighted average of this color and other color where the 
    # weight is a value between 0 and 1. 0 meaning 0% of the other color.
    assert weight <=1 and weight >= 0
    
    def linear_interpolate(c1, c2):
      return (c2 - c1) * weight + c1
    
    r = linear_interpolate(self.r, other_color.r)
    g = linear_interpolate(self.g, other_color.g)
    b = linear_interpolate(self.b, other_color.b)
    
    return rgb(r, g, b)


color0 = rgb(0, 0, 0)
color1 = rgb(132, 138, 147)
color2 = rgb(34, 157, 255)


In [3]:
def line(x, y, color, name=None):
  return go.Scatter(
    name=name,
      x = x,
      y = y,
      hoverinfo = 'text',
    mode = 'lines',
    marker={'color':str(color)},
    line_width=3,
      
  )

In [4]:
import numpy as np

In [5]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [6]:
import plotly.graph_objs as go
import plotly

In [7]:
bgcolor = "rgb(220,220,220)"

In [8]:
layout = dict(
  plot_bgcolor= bgcolor,
    width=800,
    height=1760,
 #      template = 'ggplot2',
    xaxis1 = {'domain': [0, 1], 'anchor': 'x1', 'range': [-6, 6], 'ticks':'inside', 'tick0':-6, 'dtick':2 },
    yaxis1 = {'domain': [0.8, 1], 'anchor': 'x1', 'range': [-.1, 1.1], 'ticks':'inside',  'tick0':0, 'dtick':.5},
    xaxis2 = {'domain': [0, 1], 'anchor': 'free', 'position':.04, 'range': [-2.1, 2.1], 'ticks':'inside', 'showgrid':True },
    yaxis2 = {'domain': [.04, .21], 'anchor': 'free', 'range': [-.2, 2], 'position':.00, 'showticklabels':False, 'ticks':''},
)


In [9]:
x= np.arange(-6,6,.01)
sig = np.array([sigmoid(i) for i in x])
l1 = line(x, sig, color1, name='sigmoid')
l2 = line(x, sig*(1-sig), color2, name='sigmoid gradient')
l3 = line(x, (x>0).astype(np.int), color0, name='step')
fig = go.Figure(data=[l1, l2, l3], layout=layout)

In [10]:
plotly.offline.plot(fig, filename='/home/tucker/Downloads/xor_hidden.html')

'/home/tucker/Downloads/xor_hidden.html'

In [11]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import subplots
import plotly


In [12]:
fig.write_image('../step_sigmoid_grad.png')

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    /home/tucker/.cargo/bin
    /usr/local/sbin
    /usr/local/bin
    /usr/bin
    /opt/cuda/bin
    /usr/bin/site_perl
    /usr/bin/vendor_perl
    /usr/bin/core_perl
    /home/tucker/bin
    /home/tucker/.local/bin
    /home/tucker/anaconda3/bin
    /home/tucker/bin
    /home/tucker/.local/bin
    /home/tucker/anaconda3/bin
    /opt/cuda/bin
    /usr/bin/site_perl
    /usr/bin/vendor_perl
    /usr/bin/core_perl
    /home/tucker/bin
    /home/tucker/.local/bin
    /home/tucker/anaconda3/bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orca

Alternatively, see other installation methods in the orca project README at
https://github.com/plotly/orca

After installation is complete, no further configuration should be needed.

If you have instal